In [5]:
import pandas as pd
import numpy as np
import copy
from pandas import *
import scipy.optimize as sopt
import networkx as nx
from networkx.drawing.nx_agraph import to_agraph
import matplotlib.pyplot as plt
from itertools import permutations
from scipy.optimize import bisect
np.set_printoptions(precision=3)

In [6]:
link_data = pd.read_excel('link_data.xlsx')
OD_matrix = pd.read_excel('O-D_matrix.xlsx')
link_data['cap'] = link_data['cap'] * 1000
#print(link_data)
#
#print(OD_matrix)
type(link_data['D'][0])
#print(len(link_data))

numpy.int64

## Functions defined for UE

In [7]:
def lpf_bpr(df):
    t0 = np.array(df.t0)
    ca = np.array(df.cap)
    x = np.array(df.x)
    return np.multiply(t0,(1+ 0.15*(np.power(np.divide(x,ca),4))))

def convergence(x_n,x_n1):
    return np.sqrt(np.sum(np.power(np.subtract(x_n1,x_n),2))) / np.sum(x_n)

def return_tt(df):
    x_n = np.array(df['x'])
    t0 = np.array(df.t0)
    ca = np.array(df.ca)
    obj =  np.sum(np.multiply(t0,np.add(x_n,0.15*np.divide(np.power(x_n,4),np.power(ca,4)))))
    return obj
def return_alpha_bis(alpha,df):
    obj=0
    y = np.array(df.y)
    x_n = np.array(df.x)
    t0 = np.array(df.t0)
    ca = np.array(df.cap)
    diff=y-x_n
    x_sub = np.add(x_n,alpha * np.subtract(y,x_n))
    obj += t0*(diff+0.15*diff(x_sub)**4/cap**4)
    return obj
def return_alpha(alpha,df):
    y = np.array(df.y)
    x_n = np.array(df.x)
    t0 = np.array(df.t0)
    ca = np.array(df.cap)
    x_sub = np.add(x_n,alpha * np.subtract(y,x_n))
    obj = np.sum(np.multiply(t0,np.add(x_sub,(0.15/5)*np.divide(np.power(x_sub,5),np.power(ca,4)))))
    return obj

def grad_func(x,alpha,sub,array):
    z=0.0
    dif=[]
    for i in performance_func:   ##### contains link characteristics details such free_flow,capacity etc
        t_o = performance_func[i]['free flow'] ### Free flow time
        cap = performance_func[i]['capacity'] ####capacity of the link extracted from performance_func dictionary
        j=i-1
        dif[j]=x[j]+alpha*sub[j]
        z+= t_o*(sub[j]+0.15*sub[j]*dif[j]**4/cap**4)
        return z

In [8]:
link_data['tt'] = lpf_bpr(link_data)
#print(link_data)

Shortest Path-Dijkrtas

In [9]:
def directional_network_shortest_path(link_data):
    G1=nx.DiGraph()
    G2=nx.DiGraph()
    nodes = [1,2,4,5,10,11,13,14,15,19,20,21,22,24]
    all_paths = list(permutations(nodes,2))
    #print(all_paths)
    list_nodes=[1,2,3,4,5,6,7,8,9,10,
               11,12,13,14,15,16,17,18,19,20,21,22,23,24]
    G1.add_nodes_from(list_nodes)
    G1.nodes()
    print(len(G1.nodes))
    print(len(all_paths))
    G1.add_edges_from([(link_data['O'][i],link_data['D'][i],{'dist': link_data['tt'][i]}) for i in link_data.index])
    global my_paths
    my_paths = {}
    for orig, dest in all_paths: 
            path = nx.dijkstra_path(G1, source=orig, target=dest)
            count=len(path)
            end_search=count-1
            paired_list = []
            for i in range (0,end_search):
                value_1=path[i]
                z=i+1
                value_2=path[z]
                list_pair=(value_1,value_2,)
                paired_list.append(list_pair)
                i=i+1 
                print(paired_list)
            my_paths[(orig,dest)] = paired_list
            print(f"path from {orig}->{dest} is  goes through {path}")
            



In [10]:
directional_network_shortest_path(link_data)

24
182
[(1, 2)]
path from 1->2 is  goes through [1, 2]
[(1, 3)]
[(1, 3), (3, 4)]
path from 1->4 is  goes through [1, 3, 4]
[(1, 2)]
[(1, 2), (2, 6)]
[(1, 2), (2, 6), (6, 5)]
path from 1->5 is  goes through [1, 2, 6, 5]
[(1, 3)]
[(1, 3), (3, 4)]
[(1, 3), (3, 4), (4, 11)]
[(1, 3), (3, 4), (4, 11), (11, 10)]
path from 1->10 is  goes through [1, 3, 4, 11, 10]
[(1, 3)]
[(1, 3), (3, 4)]
[(1, 3), (3, 4), (4, 11)]
path from 1->11 is  goes through [1, 3, 4, 11]
[(1, 3)]
[(1, 3), (3, 12)]
[(1, 3), (3, 12), (12, 13)]
path from 1->13 is  goes through [1, 3, 12, 13]
[(1, 3)]
[(1, 3), (3, 4)]
[(1, 3), (3, 4), (4, 11)]
[(1, 3), (3, 4), (4, 11), (11, 14)]
path from 1->14 is  goes through [1, 3, 4, 11, 14]
[(1, 3)]
[(1, 3), (3, 4)]
[(1, 3), (3, 4), (4, 11)]
[(1, 3), (3, 4), (4, 11), (11, 10)]
[(1, 3), (3, 4), (4, 11), (11, 10), (10, 15)]
path from 1->15 is  goes through [1, 3, 4, 11, 10, 15]
[(1, 2)]
[(1, 2), (2, 6)]
[(1, 2), (2, 6), (6, 8)]
[(1, 2), (2, 6), (6, 8), (8, 16)]
[(1, 2), (2, 6), (6, 8), (8

In [11]:
### In this sub-routine optimization function to be used in Frank-Wolfe Algorithm is defined 
# def opt_func(x,array):
#     z=0
#     for i in range(len(link_data)):   ##### contains link characteristics details such free_flow,capacity etc
#         t_o = link_data['t0'][i]### Free flow time
#         cap = link_data['cap'][i] ####capacity of the link extracted from performance_func dictionary
#         z+= t_o*(x[i]+0.03*x[i]**5/cap**4)
#     return z

In [12]:
### In this block Intialization is performed-after finding shortest path with free flow condition.
### For shortest paths total flow for each O-D pair is assigned to to the links on shortest path
for i in range(len(OD_matrix)):
    O_x1=OD_matrix['O'][i]
    D_x1=OD_matrix['D'][i]
    read_my_path=(O_x1,D_x1)
    for pair in my_paths[read_my_path]:
        #print(my_paths)
        for j in range(len(link_data)):
            O_x2=link_data['O'][j]
            D_x2=link_data['D'][j]
            p_comp=(O_x2,D_x2)
            if pair == p_comp:
               update_flow=OD_matrix['Q'][i]
               prev_flow=link_data['x'][j]
               new_flow=update_flow+prev_flow
               link_data.at[j,'x'] = new_flow
#print(link_data)

In [13]:
#print(link_data)

# Frank Wolfe Algorithm

In [14]:
loop_num = 1
def call_recursive_func(loop_num):
    print(f"->>>>>>>loop number-> {loop_num}")
    loop_num+=1
    link_data['tt'] = lpf_bpr(link_data)
    #### Call Djikrstra for finding the shortest path
    directional_network_shortest_path(link_data)
    ###initial y values to 0---each iteration starts with 0 and added for all path assignments
    link_data['y']=0
    print(link_data['y'])
    for i in range(len(OD_matrix)):
        O_x1=OD_matrix['O'][i]
        D_x1=OD_matrix['D'][i]
        read_my_path=(O_x1,D_x1)
    for pair in my_paths[read_my_path]:
        for j in range(len(link_data)):
            O_x2=link_data['O'][j]
            D_x2=link_data['D'][j]
            p_comp=(O_x2,D_x2)
            if pair == p_comp:
                initial_y=link_data['y'][j]
                ext_src_value=(OD_matrix['Q'][i])
                update_y_values =initial_y+ext_src_value
                link_data.at[j,'y'] = update_y_values
                #-----------------------------

    x_iter=[]
    y_iter =[]
    substr_x_y=[]
    for i in range(len(link_data)):
        x_opt=link_data['x'][i]
        x_iter.insert(i,x_opt)
        y_opt=link_data['y'][i]
        x_iter.insert(i,y_opt)
        sub_xy=y_opt-x_opt
        link_data.at[i,'y-x']=sub_xy
        substr_x_y.insert(i,sub_xy)
    ##--------------------------------
    a=0
    b=1
    alpha=(a+b)/2
    err=abs((a-b)/2)
    if err > 0.006:
        con=return_alpha_bis(alpha,link_data)
        if np.sign(con)<0:
            a=alpha
        else:
            b=alpha
        err=abs((a-b)/2)
    alpha=(a+b)/2
    alpha = minimize_scalar(return_alpha,args=(link_data), bounds=(0.0, 1.0),method='bounded').x  
    print('\tStep \t\talpha\t', np.round(alpha,decimals=6))
    update_flow=list()
    for i in range (len(link_data)):
        x_up=x_iter[i]-alpha*substr_x_y[i]
        update_flow.insert(i,x_up)
    ###-----------------convergence
    diff = 0
    prev_sum = 0.000000001
    for i in range(len(link_data)):
        diff += (link_data['x'][i] -update_flow[i] )**2
        prev_sum += link_data['x'][i]
        end_condition = diff**0.5/prev_sum
    if end_condition> 0.0000001 and loop_num<100:
        for i in range(len(link_data)):
            flow_val=update_flow[i]
            link_data.at[i,'x']=round (flow_val,5)
            
            #i=i+1
        print (link_data)
        call_recursive_func(loop_num)
    else:
        print(f'done, because diff is {end_condition}, loop num is {loop_num}!',link_data)
    

In [15]:
call_recursive_func(loop_num)

->>>>>>>loop number-> 1
24
182
[(1, 2)]
path from 1->2 is  goes through [1, 2]
[(1, 3)]
[(1, 3), (3, 4)]
path from 1->4 is  goes through [1, 3, 4]
[(1, 2)]
[(1, 2), (2, 6)]
[(1, 2), (2, 6), (6, 5)]
path from 1->5 is  goes through [1, 2, 6, 5]
[(1, 3)]
[(1, 3), (3, 4)]
[(1, 3), (3, 4), (4, 11)]
[(1, 3), (3, 4), (4, 11), (11, 10)]
path from 1->10 is  goes through [1, 3, 4, 11, 10]
[(1, 3)]
[(1, 3), (3, 4)]
[(1, 3), (3, 4), (4, 11)]
path from 1->11 is  goes through [1, 3, 4, 11]
[(1, 3)]
[(1, 3), (3, 12)]
[(1, 3), (3, 12), (12, 13)]
path from 1->13 is  goes through [1, 3, 12, 13]
[(1, 3)]
[(1, 3), (3, 4)]
[(1, 3), (3, 4), (4, 11)]
[(1, 3), (3, 4), (4, 11), (11, 14)]
path from 1->14 is  goes through [1, 3, 4, 11, 14]
[(1, 3)]
[(1, 3), (3, 4)]
[(1, 3), (3, 4), (4, 11)]
[(1, 3), (3, 4), (4, 11), (11, 10)]
[(1, 3), (3, 4), (4, 11), (11, 10), (10, 15)]
path from 1->15 is  goes through [1, 3, 4, 11, 10, 15]
[(1, 2)]
[(1, 2), (2, 6)]
[(1, 2), (2, 6), (6, 8)]
[(1, 2), (2, 6), (6, 8), (8, 16)]
[(1

TypeError: 'numpy.ndarray' object is not callable

In [152]:
link_data['tt'] = lpf_bpr(link_data)
link_data['y']=0
#print(link_data)--cell to be deleted


In [153]:
link_data['y-x']=link_data['y']-link_data['x']
#print(link_data)--cell to be deleted